# M1SD TP03 $\pi$ Monte Carlo

In [1]:
import numpy as np

In [27]:
import time

La méthode `pick_np`
 prend en entrée un ndarray de dimension `(2, n)` chaque ligne représentant respectivement les coordonnées $x,y$ d'un point dans le carré $[-1,1]$ et retourne une approximation de $\pi$ avec un masque numpy.

In [2]:
def pick_np(xxyy):
    '''
    input : xxyy ndarray de n coordonnées (x,y) dans le carre [-1, 1]
    output: count_inside nombre de coups tires dans le disque inscrit au carre
    '''
    norm = np.linalg.norm(xxyy, axis=0)
    count = np.sum(norm <= 1)
    return count

In [28]:
def pi_mc_np(n):
    '''
    input : n nombre de tirage dans le carré [-1, 1]
    output : api : valeur de pi calculée par Monte Carlo
    '''
    xxyy =np.random.uniform(-1, 1, size=(2, n))
    api = 4 * pick_np(xxyy) / n
    return api

In [23]:
n = 1_000_000_000

In [29]:
%time pi_mc_np(n)

CPU times: user 19.2 s, sys: 3.69 s, total: 22.9 s
Wall time: 23 s


3.141707456

## Splitting
1. À l’aide de numpy diviser le ndarray en sous-tableaux et à l’aide de la méthode `map` appliquer `pick_np` à chaque sous-tableau et effectuer une réduction sur l’ensemble des valeurs calculées.

In [43]:
def pi_mc_np_split(n):
    '''
    input : n nombre de tirage dans le carré [-1, 1]
    output : api : valeur de pi calculée par Monte Carlo
    '''
    xxyy =np.random.uniform(-1, 1, size=(2, n))
    l = np.split(xxyy,10,axis=1)
    tic = time.time()
    result = sum(list(map(pick_np,l)))
    toc = time.time()
    print(f"{toc - tic} seconds for {n} picks in sequential map")
    api = 4 * result / n
    return api

In [44]:
%time pi_mc_np_split(n)

6.923130989074707 seconds for 1000000000 picks in sequential map
CPU times: user 19.2 s, sys: 3.73 s, total: 22.9 s
Wall time: 23 s


3.141615036

In [ ]:
%time xxyy =np.random.uniform(-1, 1, size=(2, n))
%time res1 = 4 * pick_np(xxyy) / n
%time l = np.split(xxyy,10,axis=1)
%time res2 = 4 * sum(list(map(pick_np,l))) / n

In [ ]:
res1, res2

## Multiprocessing
1. Appliquer l’algorithme suivant en distribuant les calculs parmis les *workers* d’un `PoolProcessPoolExecutor`.
2. Mesurer les temps de restitution en variant le nombre de tir et le nombre de processus.

In [8]:
from concurrent.futures import ProcessPoolExecutor

In [32]:
def pi_mc_np_mp(n):
    '''
    input : n nombre de tirage dans le carré [-1, 1]
    output : api : valeur de pi calculée par Monte Carlo
    '''
    xxyy =np.random.uniform(-1, 1, size=(2, n))
    l = np.split(xxyy,10,axis=1)
    tic = time.time()
    with ProcessPoolExecutor(4) as p_exe:
        result = sum(list(p_exe.map(pick_np,l)))
    toc = time.time()
    print(f"{toc - tic} seconds for {n} picks in MP")
    api = 4 * result / n
    return api

In [45]:
%time pi_mc_np_mp(n)

19.295588731765747 seconds for 1000000000 picks in MP
CPU times: user 22.6 s, sys: 7.79 s, total: 30.4 s
Wall time: 35.4 s


3.141582468

## Multithreading
1. Appliquer l’algorithme suivant en distribuant les calculs parmis les *workers* d’un `ThreadPoolExecutor`.
2. Mesurer les temps de restitution en variant le nombre de tir et le nombre de thread.
3. Comparer avec les versions précédentes.

In [11]:
from concurrent.futures import ThreadPoolExecutor

In [34]:
def pi_mc_np_mt(n):
    '''
    input : n nombre de tirage dans le carré [-1, 1]
    output : api : valeur de pi calculée par Monte Carlo
    '''
    xxyy =np.random.uniform(-1, 1, size=(2, n))
    l = np.split(xxyy,10,axis=1)
    tic = time.time()
    with ThreadPoolExecutor(4) as t_exe:
        result = sum(list(t_exe.map(pick_np,l)))
    toc = time.time()
    print(f"{toc - tic} seconds for {n} picks in MT")
    api = 4 * result / n
    return api

In [46]:
%time pi_mc_np_mt(n)

2.620981216430664 seconds for 1000000000 picks in MT
CPU times: user 20.5 s, sys: 4.39 s, total: 24.9 s
Wall time: 18.7 s


3.141618232

## Optimisation
1. Modifier les méthodes suivantes afin de ne générer temporairement que les sous-tableaux.
2. Effectuer de nouvelles mesures de performance.

In [47]:
def pick_np_opt(n):
    xxyy =np.random.uniform(-1, 1, size=(2, n))
    norm = np.linalg.norm(xxyy, axis=0)
    count = np.sum(norm <= 1)
    return count

In [48]:
def pi_mc_np_opt(n):
    '''
    input : n nombre de tirage dans le carré [-1, 1]
    output : api : valeur de pi calculée par Monte Carlo
    '''
    api = 4 * pick_np_opt(n) / n
    return api

In [49]:
def pi_mc_np_split_opt(n):
    '''
    input : n nombre de tirage dans le carré [-1, 1]
    output : api : valeur de pi calculée par Monte Carlo
    '''
    l = [n//10 for i in range(10)]
    tic = time.time()
    result = sum(list(map(pick_np_opt,l)))
    toc = time.time()
    print(f"{toc - tic} seconds for {n} picks in sequential map")
    api = 4 * result / n
    return api

In [50]:
%time pi_mc_np_opt(n)

CPU times: user 19.2 s, sys: 3.69 s, total: 22.9 s
Wall time: 22.9 s


3.141568112

In [51]:
%time pi_mc_np_split_opt(n)

22.949807167053223 seconds for 1000000000 picks in sequential map
CPU times: user 19.2 s, sys: 3.69 s, total: 22.9 s
Wall time: 22.9 s


3.141672032

In [52]:
def pi_mc_np_mp_opt(n):
    '''
    input : n nombre de tirage dans le carré [-1, 1]
    output : api : valeur de pi calculée par Monte Carlo
    '''
    l = [n//10 for i in range(10)]
    tic = time.time()
    with ProcessPoolExecutor(4) as p_exe:
        result = sum(list(p_exe.map(pick_np_opt,l)))
    toc = time.time()
    print(f"{toc - tic} seconds for {n} picks in MP")
    api = 4 * result / n
    return api

In [53]:
%time pi_mc_np_mp_opt(n)

7.835677146911621 seconds for 1000000000 picks in MP
CPU times: user 4.07 ms, sys: 10.9 ms, total: 15 ms
Wall time: 7.84 s


3.14163236

In [39]:
def pi_mc_np_mt_opt(n):
    '''
    input : n nombre de tirage dans le carré [-1, 1]
    output : api : valeur de pi calculée par Monte Carlo
    '''
    l = [n//10 for i in range(10)]
    tic = time.time()
    with ThreadPoolExecutor(4) as t_exe:
        result = sum(list(t_exe.map(pick_np_opt,l)))
    toc = time.time()
    print(f"{toc - tic} seconds for {n} picks in MT")
    api = 4 * result / n
    return api

In [54]:
%time pi_mc_np_mt_opt(n)

16.91415524482727 seconds for 1000000000 picks in MT
CPU times: user 19.3 s, sys: 3.78 s, total: 23.1 s
Wall time: 16.9 s


3.141636188